# Centrality
## "Who's the Boss?"

- Definition of Centrality
- Compare and contrast popular centrality measures on dataset
    - Degree
    - Closeness
    - Betweenness
    - Eigenvector


In [74]:
import networkx as nx
import matplotlib.pyplot as plt # for plotting graphs
%matplotlib inline

In [4]:
GA = nx.read_gexf('../data/ga_graph.gexf')

## Degree Centrality

__Degree__ is a basic way of understanding a node. Intuitively, the degree of a node is the number of other nodes to which it is connected.

Let's find the degree of our main character `Grey`.

In [12]:
GA.degree("grey")

4

Likewise, we can find the degree of each cast member.

In [7]:
# See all of them (ok on this small graph)
GA.degree()

{'addison': 3,
 'adele': 1,
 'altman': 2,
 'arizona': 1,
 'avery': 1,
 'bailey': 2,
 'ben': 1,
 'chief': 2,
 'colin': 1,
 'denny': 1,
 'derek': 2,
 'ellis grey': 2,
 'finn': 1,
 'grey': 4,
 'hank': 1,
 'izzie': 4,
 'karev': 7,
 'kepner': 1,
 'lexi': 3,
 'mrs. seabury': 1,
 'nancy': 1,
 "o'malley": 4,
 'olivia': 2,
 'owen': 2,
 'preston': 1,
 'sloan': 5,
 'steve': 1,
 'susan grey': 1,
 'thatch grey': 2,
 'torres': 4,
 'tucker': 1,
 'yang': 3}

In [10]:
# Here's the top 5.
sorted(GA.degree().items(), key=lambda x:x[1], reverse=True)[:5]

[('karev', 7), ('sloan', 5), ('grey', 4), ('izzie', 4), ('torres', 4)]

While knowing the raw number is great, most centrality measures are _normalized_ between zero and one so that they can be more easily compared to one another.

For the **degree centrality** measure, the normalized interpretion is really intuitive:  

> _What percentage of nodes is this node connected to?_

Or for our Grey's Anatomy example: 

> _What percentage of the cast has this character been invovled with?_


NetworkX's degree centrality is calculated by taking the degree of the node and dividing by `n-1` where where `n` is the number of nodes in `G`.

$ {C_D (u)} = \frac{deg(u)}{{n-1}} $

⚠️ __NOTE__: `In a directed graph, both in-degree and out-degree centrality can be calculated.`

Let's calculate the degree centrality for `Grey`.

In [69]:
# Degree for the 'Grey' node
degree_grey = GA.degree("grey")  # 4 romantic partners

# Total number of nodes (excluding Grey) 
total_nodes_minus_grey = len(GA.nodes())-1  # 31 characters in the cast, excluding Grey

# Degree centrality for Grey
degree_centrality_grey = (degree_grey / total_nodes_minus_grey)
print("Calculated degree centrality for Grey:", degree_centrality_grey)

# Double check
print("Networkx degree centrality for Grey:", nx.degree_centrality(GA)["grey"])

def check_equal(val1, val2):
    assert (val1 == val2),"Centrality measure calculated incorrectly!"
    return "Values match, good job!"

check_equal(degree_centrality_grey, nx.degree_centrality(GA)["grey"])

Calculated degree centrality for Grey: 0.12903225806451613
Networkx degree centrality for Grey: 0.12903225806451613


'Values match, good job!'

Likewise, let's find the degree centrality for all characters.

In [9]:
degree_centrality = nx.degree_centrality(GA)

In [10]:
degree_centrality

{'addison': 0.0967741935483871,
 'adele': 0.03225806451612903,
 'altman': 0.06451612903225806,
 'arizona': 0.03225806451612903,
 'avery': 0.03225806451612903,
 'bailey': 0.06451612903225806,
 'ben': 0.03225806451612903,
 'chief': 0.06451612903225806,
 'colin': 0.03225806451612903,
 'denny': 0.03225806451612903,
 'derek': 0.06451612903225806,
 'ellis grey': 0.06451612903225806,
 'finn': 0.03225806451612903,
 'grey': 0.12903225806451613,
 'hank': 0.03225806451612903,
 'izzie': 0.12903225806451613,
 'karev': 0.22580645161290322,
 'kepner': 0.03225806451612903,
 'lexi': 0.0967741935483871,
 'mrs. seabury': 0.03225806451612903,
 'nancy': 0.03225806451612903,
 "o'malley": 0.12903225806451613,
 'olivia': 0.06451612903225806,
 'owen': 0.06451612903225806,
 'preston': 0.03225806451612903,
 'sloan': 0.16129032258064516,
 'steve': 0.03225806451612903,
 'susan grey': 0.03225806451612903,
 'thatch grey': 0.06451612903225806,
 'torres': 0.12903225806451613,
 'tucker': 0.03225806451612903,
 'yang': 0

In [12]:
# top 5
sorted(degree_centrality.items(), key=lambda x: x[1], reverse=True)[:5]

[('karev', 0.22580645161290322),
 ('sloan', 0.16129032258064516),
 ('grey', 0.12903225806451613),
 ("o'malley", 0.12903225806451613),
 ('torres', 0.12903225806451613)]

In [13]:
# apply measurements back to Graph
nx.set_node_attributes(GA, 'degree centrality', degree_centrality)

In [15]:
GA.node['karev']

{'degree centrality': 0.22580645161290322, 'label': 'karev'}

## Closeness Centrality
Closeness Centrality measures how many "hops" it would take to reach every other node in a network (taking the shortest path). It can be informally thought as 'average distance' to all other nodes.

In NetworkX, it the reciporical of of the *average* value, which normalizes the value in a 0 to 1 range. If you again take the reciporical of this, you'll find the *average* distance to all other nodes.

$ C_C (u) = \frac{n - 1}{\sum_{v=1}^{n-1} d(v, u)} $

⚠️ __NOTE__: `If the graph is not completely connected, this algorithm computes the closeness centrality for each connected part separately.  The closeness centrality is normalized to (n-1)/(|G|-1) where n is the number of nodes in the connected part of graph containing the node.` [[Source](https://networkx.github.io/documentation/development/_modules/networkx/algorithms/centrality/closeness.html)]

__Why should we care about closeness centrality?__

> Degree centrality measures might be criticized because they only take into account the immediate ties that an actor has, or the ties of the actor's neighbors, rather than indirect ties to all others. One actor might be tied to a large number of others, but those others might be rather disconnected from the network as a whole. In a case like this, the actor could be quite central, but only in a local neighborhood. [[Source](http://www.faculty.ucr.edu/~hanneman/nettext/C10_Centrality.html#Closeness)]

In our example, closeness centrality could perhaps help us understand which characters have the greatest potential to spread an infectous disease or STD across the cast (an admittedly dark plot arc).

Let's calculate the closeness centrality for `Grey`.  First we'll start by getting the shortest paths between `Grey` and all other characters.

In [24]:
# Shortest path between Grey and other characters
grey_shortest_path = nx.shortest_path_length(GA)['grey']
grey_shortest_path

{'addison': 2,
 'altman': 4,
 'arizona': 3,
 'avery': 5,
 'colin': 7,
 'denny': 3,
 'derek': 1,
 'finn': 1,
 'grey': 0,
 'hank': 3,
 'izzie': 2,
 'karev': 3,
 'kepner': 4,
 'lexi': 4,
 'mrs. seabury': 4,
 'nancy': 4,
 "o'malley": 1,
 'olivia': 2,
 'owen': 5,
 'preston': 7,
 'sloan': 3,
 'steve': 1,
 'torres': 2,
 'yang': 6}

In [70]:
# Sum of the shortest paths to all other characters
grey_sum_shortest_path = sum(grey_shortest_path.values())  # 77

# Closeness centrality for Grey
closeness_centrality_grey = (total_nodes_minus_grey / grey_sum_shortest_path)
print("Calculated closeness centrality for Grey:", closeness_centrality_grey)

# Double check
print("Networkx closeness centrality for Grey:", nx.closeness_centrality(GA)["grey"])

check_equal(closeness_centrality_grey, nx.closeness_centrality(GA)["grey"])

Calculated closeness centrality for Grey: 0.4025974025974026
Networkx closeness centrality for Grey: 0.2216170925848345


AssertionError: Centrality measure calculated incorrectly!

Interesting...our calculated measure is not the same as the one in NetworkX.  

_What happened here?_

This error occured because __the character relationship graph is not fully connected.__ (i.e., there are groups of characters that do not have relationships with one another).

In [102]:
# View members of different subgraphs
sorted(nx.connected_components(GA), key = len, reverse=True)

[{'addison',
  'altman',
  'arizona',
  'avery',
  'colin',
  'denny',
  'derek',
  'finn',
  'grey',
  'hank',
  'izzie',
  'karev',
  'kepner',
  'lexi',
  'mrs. seabury',
  'nancy',
  "o'malley",
  'olivia',
  'owen',
  'preston',
  'sloan',
  'steve',
  'torres',
  'yang'},
 {'adele', 'chief', 'ellis grey', 'susan grey', 'thatch grey'},
 {'bailey', 'ben', 'tucker'}]

To correct for this, we will use the number of nodes in the `Grey` subgraph instead of the total number of nodes to calculated degree centrality.  Additionally, we'll normalized to `(n-1)/(|G|-1)` where `n` is the number of nodes in the connected part of graph containing the node.

In [111]:
# Number of nodes in Grey subgraph, excluding Grey
total_nodes_minus_grey_sub = len(grey_shortest_path)-1   # 23

# Closeness centrality for Grey (unnormalized)
closeness_centrality_grey = (total_nodes_minus_grey_sub / grey_sum_shortest_path)   # ~0.2987

# Closeness centrality for Grey (normalized)
closeness_centrality_grey_normalized = closeness_centrality_grey * (total_nodes_minus_grey_sub/total_nodes_minus_grey)
print("Calculated closeness centrality for Grey (normalized):", closeness_centrality_grey_normalized)

# Double check
print("Networkx closeness centrality for Grey:", nx.closeness_centrality(GA)["grey"])

check_equal(closeness_centrality_grey_normalized, nx.closeness_centrality(GA)["grey"])

Calculated closeness centrality for Grey (normalized): 0.2216170925848345
Networkx closeness centrality for Grey: 0.2216170925848345


'Values match, good job!'

Let's find the degree centrality for all characters.

In [112]:
closeness_centrality = nx.closeness_centrality(GA)

In [113]:
closeness_centrality

{'addison': 0.2892290869327502,
 'adele': 0.05161290322580645,
 'altman': 0.2337604949182501,
 'arizona': 0.21600653327888933,
 'avery': 0.19614386355209493,
 'bailey': 0.06451612903225806,
 'ben': 0.04301075268817204,
 'chief': 0.07373271889400922,
 'colin': 0.13228307076769194,
 'denny': 0.18752215526409075,
 'derek': 0.2337604949182501,
 'ellis grey': 0.08602150537634408,
 'finn': 0.17236884978820463,
 'grey': 0.2216170925848345,
 'hank': 0.18752215526409075,
 'izzie': 0.24731182795698925,
 'karev': 0.2892290869327502,
 'kepner': 0.21067303863002787,
 'lexi': 0.26253101736972706,
 'mrs. seabury': 0.21067303863002787,
 'nancy': 0.21067303863002787,
 "o'malley": 0.2708653353814644,
 'olivia': 0.2337604949182501,
 'owen': 0.19173613628126135,
 'preston': 0.13228307076769194,
 'sloan': 0.2892290869327502,
 'steve': 0.17236884978820463,
 'susan grey': 0.05161290322580645,
 'thatch grey': 0.07373271889400922,
 'torres': 0.29937747594793435,
 'tucker': 0.04301075268817204,
 'yang': 0.15948

In [20]:
# top 5
sorted(closeness_centrality.items(), key=lambda x: x[1], reverse=True)[:5]

[('torres', 0.29937747594793435),
 ('addison', 0.2892290869327502),
 ('sloan', 0.2892290869327502),
 ('karev', 0.2892290869327502),
 ("o'malley", 0.2708653353814644)]

In [23]:
# apply measurements back to Graph
nx.set_node_attributes(GA, 'closeness centrality', closeness_centrality)

In [25]:
# average distance of torres:
1 / GA.node['torres']['closeness centrality']

3.340264650283554

In [15]:
7/(len(GA.nodes()) - 1)

0.22580645161290322

## Betweeness Centrality

Betweenness centrality quantifies the number of times a node acts as a bridge along the shortest path between two other nodes.  In his conception, vertices that have a high probability to occur on a randomly chosen shortest path between two randomly chosen vertices have a high betweenness.

$ C_B(v) =\sum_{s,t \in V} \frac{\sigma(s, t|v)}{\sigma(s, t)} $

where ${\sigma(s, t)}$ is total number of shortest paths from node ${s}$ to node ${t}$ and ${\sigma(s, t|v)}$ is the number of those paths that pass through ${v}$.

__Why is betweeness centrality important?__

> Suppose that I want to influence you by sending you information, or make a deal to exchange some resources. But, in order to talk to you, I must go through an intermediary. For example, let's suppose that I wanted to try to convince the Chancellor of my university to buy me a new computer. According to the rules of our bureaucratic hierarchy, I must forward my request through my department chair, a dean, and an executive vice chancellor. Each one of these people could delay the request, or even prevent my request from getting through. This gives the people who lie "between" me and the Chancellor power with respect to me... Having more than one channel makes me less dependent, and, in a sense, more powerful.  [[Source](http://www.faculty.ucr.edu/~hanneman/nettext/C10_Centrality.html#Closeness)]

While this metric seems less relevent to our particular network, let's create a hypothetical situation:

To engage with a new romantic partner, you needed permission from at least one of their former partners and you could only send your request through existing partners (_awkward_ 😬).  Betweeness centrality would tell you which actors had the most effective veto power to thwart random requests made by a member to another random member in the network.

In [162]:
actors         = nx.nodes(GA)
shortest_paths = nx.shortest_path(GA)
connect_set    = set()
numerator      = 0
denominator    = 0

for a in actors:
    char_shortest_paths = shortest_paths[a]  # Gets a character's list of shortest paths to other characters
    for end,path in char_shortest_paths.items():
        connect_set.add(a,end)
        denominator+=1
        if 'grey' in path:
            numerator+=1

In [115]:
betweeness_centrality = nx.betweenness_centrality(GA)

In [116]:
betweeness_centrality

{'addison': 0.09480286738351255,
 'adele': 0.0,
 'altman': 0.16344086021505377,
 'arizona': 0.0,
 'avery': 0.0,
 'bailey': 0.002150537634408602,
 'ben': 0.0,
 'chief': 0.0064516129032258064,
 'colin': 0.0,
 'denny': 0.0,
 'derek': 0.038602150537634404,
 'ellis grey': 0.008602150537634409,
 'finn': 0.0,
 'grey': 0.10078853046594984,
 'hank': 0.0,
 'izzie': 0.10311827956989247,
 'karev': 0.20487455197132612,
 'kepner': 0.0,
 'lexi': 0.07741935483870968,
 'mrs. seabury': 0.0,
 'nancy': 0.0,
 "o'malley": 0.11702508960573477,
 'olivia': 0.01064516129032258,
 'owen': 0.12903225806451613,
 'preston': 0.0,
 'sloan': 0.24810035842293907,
 'steve': 0.0,
 'susan grey': 0.0,
 'thatch grey': 0.0064516129032258064,
 'torres': 0.14440860215053763,
 'tucker': 0.0,
 'yang': 0.09247311827956989}

In [117]:
# top 5
sorted(betweeness_centrality.items(), key=lambda x: x[1], reverse=True)[:5]

[('sloan', 0.24810035842293907),
 ('karev', 0.20487455197132612),
 ('altman', 0.16344086021505377),
 ('torres', 0.14440860215053763),
 ('owen', 0.12903225806451613)]

## Eigenvector Centrality

A node is high in eigenvector centrality if it is connected to many other nodes who are themselves well connected. A node's centrality is dependent on the centrality of adjacent nodes. These nodes would be considered influential. Eigenvector centrality for each node is simply calculated as the proportional eigenvector values of the eigenvector with the largest eigenvalue.

In [226]:
eigenvector_centrality = nx.eigenvector_centrality_numpy(GA)

In [227]:
eigenvector_centrality

{'addison': 0.2784013959452947,
 'adele': 9.751217751879261e-17,
 'altman': 0.10442628192357345,
 'arizona': 0.1056420154369081,
 'avery': 0.07734385472828517,
 'bailey': -9.129231951768135e-17,
 'ben': 1.659967312204627e-17,
 'chief': 1.7908035689356888e-16,
 'colin': 0.00352438973481364,
 'denny': 0.08320301854301726,
 'derek': 0.12570740328311858,
 'ellis grey': 1.1938517495344422e-16,
 'finn': 0.04422017135181146,
 'grey': 0.15107836088557375,
 'hank': 0.08320301854301713,
 'izzie': 0.28426338654827693,
 'karev': 0.5027687871890414,
 'kepner': 0.1471588769531375,
 'lexi': 0.26424553410154455,
 'mrs. seabury': 0.1471588769531374,
 'nancy': 0.09444834886225352,
 "o'malley": 0.3020119709505962,
 'olivia': 0.23555685153699457,
 'owen': 0.03408964112637767,
 'preston': 0.0035243897348136385,
 'sloan': 0.322683094575425,
 'steve': 0.04422017135181152,
 'susan grey': 4.009896806784398e-17,
 'thatch grey': 3.852085892464765e-17,
 'torres': 0.3609262932492623,
 'tucker': 1.9494846887876484e

In [225]:
maxval = 0
maxkey = ''
for k in eigenvector_centrality.keys():
    if eigenvector_centrality[k] > maxval:
        maxval = eigenvector_centrality[k]
        maxkey = k
        
print("maxkey:", maxkey, ", maxval:", maxval)

ec_scaled = {}
for k in eigenvector_centrality.keys():
    ec_scaled[k] = eigenvector_centrality[k] / maxval

# print ec_scaled # display
sorted(ec_scaled.items(), key=lambda x:x[1], reverse=True)[0:5]

maxkey: karev , maxval: 0.5027687871890412


[('karev', 1.0),
 ('torres', 0.7178772876239705),
 ('sloan', 0.6418121068722915),
 ("o'malley", 0.600697534624479),
 ('izzie', 0.5653958515157257)]

In [221]:
    import scipy as sp
    from scipy.sparse import linalg
    M = nx.to_scipy_sparse_matrix(GA, nodelist=GA.nodes(), dtype=float)
    eigenvalue, eigenvector = linalg.eigs(M.T, k=1, which='LR')  # Get largest real part
    largest = eigenvector.flatten().real
    norm = sp.sign(largest.sum())*sp.linalg.norm(largest)
    centrality = dict(zip(GA,map(float,largest/norm)))
    centrality

{'addison': 0.2784013959452948,
 'adele': -6.216405248694224e-18,
 'altman': 0.10442628192357337,
 'arizona': 0.10564201543690806,
 'avery': 0.07734385472828523,
 'bailey': 2.1719437361702516e-18,
 'ben': 2.621488290874273e-17,
 'chief': 3.407914118028536e-17,
 'colin': 0.0035243897348135687,
 'denny': 0.08320301854301713,
 'derek': 0.12570740328311858,
 'ellis grey': -1.0580088572146739e-17,
 'finn': 0.04422017135181141,
 'grey': 0.1510783608855733,
 'hank': 0.08320301854301713,
 'izzie': 0.2842633865482764,
 'karev': 0.5027687871890417,
 'kepner': 0.14715887695313754,
 'lexi': 0.2642455341015449,
 'mrs. seabury': 0.14715887695313767,
 'nancy': 0.0944483488622535,
 "o'malley": 0.30201197095059595,
 'olivia': 0.23555685153699427,
 'owen': 0.034089641126377634,
 'preston': 0.0035243897348135682,
 'sloan': 0.32268309457542516,
 'steve': 0.04422017135181143,
 'susan grey': 1.9401180607718834e-18,
 'thatch grey': 1.722879985362252e-17,
 'torres': 0.360926293249262,
 'tucker': 3.56294630071